In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from nlp_embedding import GloVe
from nlp_classifier import NaiveBayes, SVM, XGBoost

In [ ]:
if not os.path.exists('models'):
    os.makedirs('models')

if not os.path.exists('predictions'):
    os.makedirs('predictions')

In [ ]:
def train(data_x, data_y, nlp_embedding, nlp_classifier, batch_size=100, model_dir='models', start_idx=0):
    print('Training...')
    classes = np.unique(data_y)
    
    for i in range(start_idx, data_x.shape[0], batch_size):
        
        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        nlp_classifier.partial_fit(embeddings, data_y[i:j], classes=classes)
        nlp_classifier.save(os.path.join(model_dir, f'model_{nlp_embedding.name}_{nlp_classifier.name}.joblib'))
    
    print('Success!')

In [ ]:
def test(data_x, nlp_embedding, nlp_classifier, batch_size=100, predictions_dir='predictions', start_idx=0):
    print('Testing...')
    fname = os.path.join(predictions_dir, f'model_{nlp_embedding.name}_{nlp_classifier.name}.csv')
    predictions_all = []

    if start_idx == 0 and os.path.exists(fname):
        os.remove(fname)
    
    for i in range(start_idx, data_x.shape[0], batch_size):

        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        predictions = nlp_classifier.predict(embeddings)
        
        predictions_all.extend(predictions)

        pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    
    print('Success!')    
    
    return predictions_all

In [ ]:
max_words = 400

In [ ]:
train_data = pd.read_csv('data/train/metrolyrics.csv')
test_data = pd.read_csv('data/test/metrolyrics.csv')

In [ ]:
nlp_classifier = NaiveBayes()
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier)
p = test(test_data.lyrics, nlp_embedding, nlp_classifier)
accuracy_score(test_data.genre.values, p)

In [ ]:
nlp_classifier = SVM()
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier)
p = test(test_data.lyrics, nlp_embedding, nlp_classifier)
accuracy_score(test_data.genre.values, p)

In [ ]:
nlp_classifier = XGBoost()
nlp_embedding = GloVe(max_words)
train(train_data.lyrics, train_data.genre, nlp_embedding, nlp_classifier)
p = test(test_data.lyrics, nlp_embedding, nlp_classifier)
accuracy_score(test_data.genre.values, p)